In [ ]:
import requests
import oauth2
import json
import pickle 
import os

with open("twitter_auth.json", 'r') as f:
    auth_codes = json.load(f)
    
def load_pickle(fname):
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    return data
    
def add_to_pickle_file(fname, new_data):
    # Given a pickle filename, append to it the new data (in our case, a list of dicts)
   
    # Check that file is not empty, otherwise initialize
    if os.path.getsize(fname) > 0:      
        with open(fname, 'rb') as f:
            unpickler = pickle.Unpickler(f);
            data = unpickler.load()
    else:
        data = []
    
    print("Old length of file: {:d}".format(len(data)))
    data = data + new_data
    print("New length of file: {:d}".format(len(data)))
    
    with open(fname, 'wb') as f:
        pickle.dump(data, f)

def oauth_req(url, http_method="GET", post_body=b"", http_headers=None):
    consumer = oauth2.Consumer(key=auth_codes["key"], secret=auth_codes["secret"])
    token = oauth2.Token(key="", secret="")
    client = oauth2.Client(consumer, token)
    resp, content = client.request( url, method=http_method, body=post_body, headers=http_headers )
    return content

byteResponse = oauth_req('https://api.twitter.com/1.1/search/tweets.json?q=a&lang=en&count=100&tweet_mode=extended', "GET")
tweets = json.loads(byteResponse)

pickle_file = "ben_labels.pckl"
# pickle_file = "patrick_labels.pckl"

In [ ]:
# Usage: 
# 0) Make empty file called "patrick_labels.pckl" in working directory
# 1) Select correct pickle file
# 2) Label tweets as appropriate
# 3) When finished, type "done" into the emotion box to automatically have tweets uploaded to the pickle file. 


labeledTweets = []

print("Total number of tweets to analyze in this session: {:d}".format(len(tweets['statuses'])))

# p - positive, n - negative, i - informational, h - humor, a - advertisement, o - none of the above
for status in tweets['statuses']:
    text = status['full_text']
    # If this is a retweet, the full_text field gets cut off at 140 characters, 
    # so we have to get the whole text from the retweeted_status field
    if text[0:4] == 'RT @':
        text = status['retweeted_status']['full_text']
    print(text)
    emotion = input("What emotion is this? (type 'done' to save current progress to file) ")
    print("\n")
    if emotion.lower() == "done":
        break
    tweet = {"status": text, "userId": status['user']['id'], "emotion": emotion}
    labeledTweets.append(tweet)

add_to_pickle_file(pickle_file, labeledTweets)
print("Added {:d} datapoints".format(len(labeledTweets)))

In [ ]:
tweets['statuses'][1]['retweeted_status']['full_text']

In [ ]:
labeledTweets

In [ ]:
#transfer txt file to pickle file
import ast

f = open("patrickClassified.txt",'r')
lines = f.readlines()
f.close()

dictString = ""
inStatus = False
tweets = []
for i, line in enumerate(lines):
    if "\'emotion\'" in line:
        if "[" in line:
            ind = line.index("[") + 1
            line = line[ind:]
        dictString = dictString + line
    elif "\'status\'" in line:
        inStatus = True
        dictString = dictString + line
    elif "\'userId\'" in line:
        inStatus = False
        if "," in line:
            ind = line.index(",")
            line = line[:ind]
        elif "]" in line:
            ind = line.index("]")
            line = line[:ind]
        dictString = dictString + line
        tweet = ast.literal_eval(dictString.lstrip())
        tweets.append(tweet)
        dictString = ""
    elif inStatus:
        dictString = dictString + line

#add_to_pickle_file("patrick_labels.pckl", tweets)
        

In [ ]:
# Experimenting to see what features will work best

# Feature ideas: length of status, presence of url, number of @'s, number of #'s, presence/number of emojis, 
# number of numeric/special characters as opposed to alpha, wordnet to assess sentiment, number of alpha words that
# are not real dictionary words

def statusLength(status):
    return len(status)

def hasLink(status):
    if "https://" in status:
        return 1
    else:
        return 0
    
def numAts(status):
    return status.count("@")


patrickData = load_pickle("patrick_labels.pckl")
#patrickData = patrickData[:270]
#patrickData = patrickData[270:540]
patrickData = patrickData[540:]
#benData = load_pickle("ben_labels.pckl")

allCategories = ["p","n","h","a","i","o"]

allStatuses = {}
for category in allCategories:
    allStatuses[category] = []
for tweet in patrickData:
    categories = tweet['emotion']
    for category in categories:
        allStatuses[category].append(tweet['status'])
        
features = [statusLength, hasLink, numAts]

for category in allCategories:
    print("Category: " + category)
    statuses = allStatuses[category]
    numStatuses = len(statuses)
    print(numStatuses)
    totals = {}
    for feature in features:
        totals[feature] = 0
    for status in statuses:
        for feature in features:
            totals[feature] += feature(status)
    for feature in features:
        print(totals[feature]/numStatuses)
    
    
    

In [ ]:
data = load_pickle(pickle_file)

def make_features(data):
    import nltk
    
    statuses = [d['status'] for d in data]
    labels = [d['emotion'] for d in data]
    
    return statuses, labels
        
print(data[0])